# 자율적인 AI - Agent & Tools (에이전트와 도구)

LLM이 단순히 텍스트만 생성하는 것이 아니라, **도구(Tools)**를 사용하여 검색, 계산, API 호출 등 실질적인 행동(Action)을 결정하고 수행하는 **Agent**를 실습합니다.

**학습 목표:**
1. **Tools (도구):** 검색(Tavily, DDG), 위키피디아 등 외부 정보를 가져오는 도구 활용하기
2. **Custom Tool:** 나만의 파이썬 함수를 도구로 만들어 에이전트에 능력 부여하기
3. **Agent Executor:** LLM이 스스로 판단(Reasoning) -> 도구 실행(Action) -> 결과 관찰(Observation) -> 최종 답변(Answer)을 반복하는 에이전트 런타임 구성하기

### 1. 환경 설정 (Environment Setup)

In [ ]:
%pip install langchain_openai langchain-community langchain-tavily wikipedia api numexpr duckduckgo-search -Uqqq

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")

# LangSmith 설정
os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")

---### 2. Search Tools (검색 도구)

LLM은 학습 시점 이후의 정보를 모릅니다. 검색 도구를 쥐어주면 최신 정보를 찾아 답변할 수 있습니다.
- `Tavily`: AI에 최적화된 검색 엔진 (강력 추천)
- `DuckDuckGo`: 무료 검색 엔진

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

# 1. Tavily 검색 도구
tavily_tool = TavilySearchResults(k=3)

# 2. DuckDuckGo 검색 도구
ddg_tool = DuckDuckGoSearchRun()

# 3. Wikipedia 도구
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 도구 테스트
print("--- Tavily 검색 테스트 ---")
print(tavily_tool.invoke("LangChain 최신 버전은?"))

---### 3. Custom Tool (사용자 정의 도구)

`@tool` 데코레이터를 사용하여 파이썬 함수를 에이전트가 사용할 수 있는 도구로 변환합니다.
**Docstring(함수 설명)**을 상세히 작성해야 LLM이 언제 어떻게 이 도구를 쓸지 잘 판단합니다.

In [ ]:
from langchain_core.tools import tool

@tool
def complicated_calculation(a: int, b: int) -> int:
    """두 숫자를 더한 후 10배를 곱하는 복잡한 계산을 수행합니다."""
    return (a + b) * 10

@tool
def get_weather(city: str) -> str:
    """특정 도시의 현재 날씨를 가져옵니다. (Mock 데이터)"""
    if "서울" in city:
        return "맑음, 25도"
    elif "런던" in city:
        return "비, 15도"
    else:
        return "알 수 없음, 20도"

# 도구 목록 리스트
tools = [tavily_tool, complicated_calculation, get_weather]

---### 4. Agent Executor 구성

`create_tool_calling_agent`: LLM이 도구 호출(Tool Calling) 기능을 사용해 에이전트 로직을 수행하도록 연결합니다.
`AgentExecutor`: 에이전트의 반복 실행(생각 -> 도구실행 -> 결과확인)을 관리하는 런타임입니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# 1. LLM 정의 (gpt-4o-mini 등 Tool Calling 지원 모델 권장)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유용한 AI 어시스턴트입니다. 질문에 답하기 위해 필요한 도구를 사용하세요."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")  # 에이전트의 생각/도구실행 기록이 들어갈 자리
])

# 3. 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# 4. 에이전트 실행기(Executor) 생성
# verbose=True: 에이전트의 사고 과정을 출력해서 보여줌
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. 실행 테스트
print("--- Test 1: 일반 상식 (검색 불필요/필요 시 검색) ---")
agent_executor.invoke({"input": "LangChain이 뭐야?"})

print("\n--- Test 2: Custom Tool (날씨) ---")
agent_executor.invoke({"input": "지금 서울 날씨 어때?"})

print("\n--- Test 3: Custom Tool (계산) ---")
agent_executor.invoke({"input": "5와 3을 더해서 10배 하면 몇이야? (복잡한 계산기 써줘)"})